In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

import warnings
warnings.filterwarnings('ignore')


In [3]:
data_path = 'data/'
df_train = pd.read_csv(data_path + 'train_data.csv')

df_test = pd.read_csv(data_path + 'test_features.csv')

df_train.head()

,name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,RICE KENNETH D,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,...,1617011.0,174839.0,True,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,SKILLING JEFFREY K,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,...,1920000.0,22122.0,True,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,SHELBY REX,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,...,NaN,1573324.0,True,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,KOPPER MICHAEL J,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,...,602671.0,907502.0,True,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,CALGER CHRISTOPHER F,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,...,375304.0,486.0,True,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [4]:
df_train.describe()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
count,6.100000e+01,2.800000e+01,3.400000e+01,13.000000,8.100000e+01,73.000000,65.000000,65.000000,65.000000,2.000000e+00,4.900000e+01,6.900000e+01,8.200000e+01,10.000000,7.300000e+01,65.000000,65.000000,9.600000e+01,9.800000e+01
mean,1.147436e+06,6.344374e+05,-4.625664e+05,89397.846154,2.985081e+06,51040.547945,711.323077,64.800000,40.092308,4.096250e+07,7.926171e+05,4.471774e+05,1.294855e+06,-221885.700000,2.739025e+05,1111.369231,2156.061538,2.590977e+06,3.527136e+06
std,1.505189e+06,8.603646e+05,8.095392e+05,41143.391399,6.004174e+06,47596.682104,2074.497628,91.863214,88.901407,5.736404e+07,9.504645e+05,1.341564e+06,2.498335e+06,205191.374121,1.716647e+05,1165.852016,2811.676718,1.056645e+07,7.182997e+06
min,7.000000e+04,-1.025000e+05,-3.504386e+06,3285.000000,3.285000e+03,148.000000,12.000000,0.000000,0.000000,4.000000e+05,7.102300e+04,2.000000e+00,4.409300e+04,-560222.000000,4.770000e+02,2.000000,57.000000,1.480000e+02,-4.409300e+04
25%,4.500000e+05,7.656750e+04,-5.527032e+05,101250.000000,4.004780e+05,18834.000000,19.000000,10.000000,0.000000,2.068125e+07,2.750000e+05,9.720000e+02,2.689220e+05,-389621.750000,2.061210e+05,178.000000,517.000000,3.024025e+05,4.211518e+05
50%,7.500000e+05,1.951900e+05,-1.175340e+05,108579.000000,8.500100e+05,41953.000000,45.000000,28.000000,7.000000,4.096250e+07,4.221580e+05,5.238200e+04,4.628225e+05,-139856.500000,2.516540e+05,599.000000,1088.000000,1.106740e+06,9.979710e+05
75%,1.000000e+06,8.342052e+05,-2.708325e+04,112492.000000,2.165172e+06,59175.000000,215.000000,88.000000,27.000000,6.124375e+07,8.318090e+05,3.620960e+05,9.664905e+05,-77953.250000,2.885890e+05,1902.000000,2649.000000,1.985668e+06,2.493616e+06
max,8.000000e+06,2.964506e+06,-1.042000e+03,125034.000000,3.434838e+07,228763.000000,14368.000000,528.000000,411.000000,8.152500e+07,5.145434e+06,1.035973e+07,1.476169e+07,44093.000000,1.111258e+06,4527.000000,15149.000000,1.035598e+08,4.911008e+07


In [4]:
df_train['poi'] = df_train['poi'].astype(float)
train_Y = df_train['poi']
ids = df_test['name']

df_train = df_train.drop(['name', 'email_address', 'poi'] , axis=1)
df_test = df_test.drop(['name', 'email_address'] , axis=1)

df = pd.concat([df_train,df_test])
df = df.fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146 entries, 0 to 32
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   bonus                      146 non-null    float64
 1   deferral_payments          146 non-null    float64
 2   deferred_income            146 non-null    float64
 3   director_fees              146 non-null    float64
 4   exercised_stock_options    146 non-null    float64
 5   expenses                   146 non-null    float64
 6   from_messages              146 non-null    float64
 7   from_poi_to_this_person    146 non-null    float64
 8   from_this_person_to_poi    146 non-null    float64
 9   loan_advances              146 non-null    float64
 10  long_term_incentive        146 non-null    float64
 11  other                      146 non-null    float64
 12  restricted_stock           146 non-null    float64
 13  restricted_stock_deferred  146 non-null    float64


In [5]:
#finalcial 和 email 數據的數值資訊都捨去, 有就是1, 沒有(na)就是0
for col in df:
    df[col] = df[col].apply(lambda x: 1 if abs(x)>0 else 0)

#先前測試, 發現有時候把 'bonus' 和 'deferred_income' 結合成新的 feature 結果比較好
df ['com2'] = df['bonus'] + df['deferred_income']
df.drop(['bonus', 'deferred_income'], axis = 1)
        
df.describe()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value,com2
count,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000
mean,0.561644,0.267123,0.335616,0.116438,0.698630,0.650685,0.589041,0.506849,0.452055,0.027397,0.452055,0.636986,0.753425,0.123288,0.650685,0.589041,0.589041,0.856164,0.863014,0.897260
std,0.497894,0.443980,0.473831,0.321854,0.460433,0.478395,0.493701,0.501674,0.499409,0.163800,0.499409,0.482524,0.432501,0.329899,0.478395,0.493701,0.493701,0.352131,0.345016,0.749524
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000


In [9]:
#測試幾種model後, 發現 gdbt 和 clf 分的比較好
from sklearn.metrics import accuracy_score
gdbt = GradientBoostingClassifier(tol=100, subsample=0.75, n_estimators=250, max_features=9,
                                  max_depth=6, learning_rate=0.03)
         
train_num = train_Y.shape[0]
train_X = df[:train_num]
test_X = df[train_num:]
            
X_train, X_test, y_train, y_test = train_test_split(train_X, train_Y, test_size=0.25, random_state=4)

          
gdbt.fit(train_X, train_Y)
gdbt_pred = gdbt.predict_proba(test_X)[:,1]
gdbt_score = cross_val_score(gdbt, train_X, train_Y, cv=5).mean()

#sub = pd.DataFrame({'name': ids, 'poi': gdbt_pred})
#sub.to_csv('mid_abs_com2_gdbt.csv', index=False)
            

# 建立模型
clf = DecisionTreeClassifier()
clf.fit(train_X, train_Y)
clf_pred = clf.predict_proba(test_X)[:,1]
clf_score = cross_val_score(clf, train_X, train_Y, cv=5).mean()

#sub = pd.DataFrame({'name': ids, 'poi': clf_pred})
#sub.to_csv('mid_abs_com2_clf.csv', index=False)


print('gdbt:')
print(gdbt_score)

print('clf:')
print(clf_score)            



gdbt:
0.859288537549407
clf:
0.867588932806324


In [10]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

tree_params = {"criterion":['gini', 'entropy']
               ,"max_depth": [*range(5,10)]
               , "min_samples_leaf": [*range(4,10)]
               , "min_samples_split": [*range(3,9)]
               , "random_state" :[200]}

def get_best_parameter(model, params, X, y):
    grid = RandomizedSearchCV(model, params, cv=5, verbose=1, scoring="accuracy", n_iter=300, n_jobs = -1)  
    grid.fit(X, y)
    print(f"Best accuracy: {grid.best_score_}")
    print(f"Best parameters: {grid.best_params_}")
    print(f"Average time(s) to fit: {round(grid.cv_results_['mean_fit_time'].mean(), 3)}")
    print(f"Average time(s) to score: {round(grid.cv_results_['mean_score_time'].mean(), 3)}")
    #means = grid.cv_results_['mean_test_score']
    #stds = grid.cv_results_['std_test_score']
    #  這邊依設置的參數列印出所有參數的得分狀況
    #for mean, std, params in zip(means, stds, grid.cv_results_['params']):
        #print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    return grid
tree = DecisionTreeClassifier()
tree = get_best_parameter(tree, tree_params, train_X, train_Y)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.9s


Best accuracy: 0.8853754940711462
Best parameters: {'random_state': 200, 'min_samples_split': 8, 'min_samples_leaf': 9, 'max_depth': 6, 'criterion': 'entropy'}
Average time(s) to fit: 0.005
Average time(s) to score: 0.003


[Parallel(n_jobs=-1)]: Done 1392 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:    6.5s finished


In [13]:
clf_score_best = cross_val_score(tree, train_X, train_Y, cv=5).mean()
clf_score_best

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done 1384 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:   14.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 1424 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:    1.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 300 candidates, totalling 1500 fits


...

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 1424 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:    1.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 1424 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:    1.6s finished


0.859288537549407

In [20]:
from sklearn.model_selection import GridSearchCV
def get_best_parameter2(model, params, X, y):
    grid = GridSearchCV(model, params, cv=5, verbose=1, scoring="accuracy", n_jobs=-1)  
    grid.fit(X, y)
    print(f"Best accuracy: {grid.best_score_}")
    print(f"Best parameters: {grid.best_params_}")
    print(f"Average time(s) to fit: {round(grid.cv_results_['mean_fit_time'].mean(), 3)}")
    print(f"Average time(s) to score: {round(grid.cv_results_['mean_score_time'].mean(), 3)}")
    #means = grid.cv_results_['mean_test_score']
    #stds = grid.cv_results_['std_test_score']
    #  這邊依設置的參數列印出所有參數的得分狀況
    #for mean, std, params in zip(means, stds, grid.cv_results_['params']):
        #print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    return grid

In [ ]:
(tol=100, subsample=0.75, n_estimators=250, max_features=9,
                                  max_depth=6, learning_rate=0.03)

In [21]:
#from scipy.stats import uniform as sp_randFloat
#from scipy.stats import randint as sp_randInt

model = GradientBoostingClassifier()
parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 5),
    "min_samples_leaf": np.linspace(0.1, 0.5, 5),
    "max_depth":[3,5,8],
    "subsample":[0.5, 0.618, 0.8, 0.85],
    "n_estimators":[200,250,300]
    }
clf = get_best_parameter2(model , parameters, train_X, train_Y)
yy = clf.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': ids, 'poi': yy})
sub.to_csv('sub_gdbt.csv', index=False)

Fitting 5 folds for each of 6300 candidates, totalling 31500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   17.3s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   24.9s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   33.8s
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:   46.7s
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 8434 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 9784 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 11234 tasks      |

Best accuracy: 0.9027667984189722
Best parameters: {'learning_rate': 0.025, 'loss': 'deviance', 'max_depth': 3, 'min_samples_leaf': 0.1, 'min_samples_split': 0.1, 'n_estimators': 250, 'subsample': 0.5}
Average time(s) to fit: 0.178
Average time(s) to score: 0.004


In [15]:
gdbt_score_best = cross_val_score(gradient_tree, train_X, train_Y, cv=5).mean()
gdbt_score_best

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   37.9s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:  2.3min finished


Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 200 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done 450 tasks      | elapsed:   41.7s
[Parallel(n_jobs=-1)]: Done 800 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 1250 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:  2.4min finished


Fitting 5 folds for each of 300 candidates, totalling 1500 fits


...

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   17.6s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   42.9s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:  2.5min finished


Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   18.6s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   40.7s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:  2.6min finished


0.850197628458498

In [11]:
from xgboost import XGBClassifier
gbm_param_grid = {
    'n_estimators':range(80,200,4), # 樹有幾棵
    'max_depth':range(2,15,1), # 樹的深度
    'learning_rate':np.linspace(0.01,0.08,20),
    'colsample_bytree':np.linspace(0.5,0.98,10),
    'min_child_weight':range(1,9,1),
    'subsample':np.linspace(0.7,0.9,20)
    }

XGB_tree = XGBClassifier()
XGB_tree = get_best_parameter(gradient_tree , gbm_param_grid, train_X, train_Y)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   18.0s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   28.2s


[23:24:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best accuracy: 0.8940711462450592
Best parameters: {'subsample': 0.8368421052631578, 'n_estimators': 88, 'min_child_weight': 2, 'max_depth': 12, 'learning_rate': 0.05789473684210527, 'colsample_bytree': 0.82}
Average time(s) to fit: 0.173
Average time(s) to score: 0.004


[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:   34.0s finished


In [17]:
yy = tree.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': ids, 'poi': yy})
sub.to_csv('sub_clf.csv', index=False)